## Data Engineering Capstone Project

### Project Summary

#### Introduction

A core responsibility of The National Travel and Tourism Office (NTTO) is to collect, analyze, and disseminate international travel and tourism statistics. 

NTTO's Board of Managers are charged with managing, improving, and expanding the system to fully account and report the impact of travel and tourism in the United States. The analysis results help to forcecast and operation, support make decision creates a positive climate for growth in travel and tourism by reducing institutional barriers to tourism, administers joint marketing efforts, provides official travel and tourism statistics, and coordinates efforts across federal agencies.

#### Project Description
The target of project is analysis the relationship between amount of travel immigration and weather duration by month of city.

In this project, some source datas will be use to do data modeling:
- `I94 Immigration` The source data for I94 immigration data is available in local disk in the format of sas7bdat. This data comes from US National Tourism and Trade Office. The data dictionary is also included in this project for reference. The actual source of the data is from https://travel.trade.gov/research/reports/i94/historical/2016.html. This data is already uploaded to the workspace.

- `World Temperature Data` This dataset came from Kaggle. This data is already uploaded to the workspace.

- `I94_SAS_Labels_Descriptions.SAS` to get validation dataset. We will use `I94Port.txt` as list of airport, city, state.

#### The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

##### Scope 

To make decision of project scope and the technical step solution we do data assessment on datasets:
* `I94 Immigration`
* `World Temperature Data`
* `I94_SAS_Labels_Descriptions.SAS`

Technologies will be used:
- Spark, Spark SQL
- Python, Pandas
- AWS S3

##### Describe and Gather Data

- I94 Immigration dataset:
    - `cicid`: Visitor US cid code issue on every travller get throught the immigration port.
    - `i94yr | i94mon`: Year, Month of immigration date.
    - `i94cit | i94res`: Country of Citizenship & Country of recidence. From `I94_SAS_Labels_Descriptions.SAS`. Don't use.
    - `i94port`: Port code for a specific immigration port USA city. From `I94_SAS_Labels_Descriptions.SAS`. There are types of airport that do not allow immigration entry.
    - `arrdate | depdate`: Arrival date in the USA & Departure date from the USA.
    - `i94mode`: Code for immigration transportation mode. From `I94_SAS_Labels_Descriptions.SAS`. There are methods of immigration transportation without airport gateway. Don't use.
    - `i94addr`: US state code. From `I94_SAS_Labels_Descriptions.SAS`.
    - `i94bir`: Age of traveller in Years. Don't use.
    - `i94visa`: Code for visa type corresponse to visiting reason. Don't use.
    - `count, tadfile, visapost, occup, entdepa, entdepd, entdepu, matflag, dtaddto, insnum`: Don't use.
    - `biryear`: Immigrant year of birth. Have to review data type. Don't use.
    - `gender`: Immigrant sex. There are some un-common sex kind. Don't use.
    - `airline`: Airline Coporate used to arrive in U.S. Don't use.
    - `admnum`: Admission Number. Don't use.
    - `fltno`: Flight number of Airline used to arrive in U.S. Don't use.
    - `visatype`: Class of admission legally admitting the non-immigrant to temporarily stay in U.S. Don't use.


- World Temperature dataset
    - `dt`: The creation time of temperature.
    - `AverageTemperature | AverageTemperatureUncertainty`: temperature value recognized.
    - `City | Country`: City of Country that the teperature recognized.
    - `Latitude | Longitude`: Geographical location in lat-long. Helpful for heatmap but these columns is useless in our project.

- I94_SAS_Labels_Descriptions.SAS contains information parts:
    - `I94PORT_sas_label_validation` (to be used later)
        - `i94port_valid_code`: airport code.
        - `i94port_city_name`: the city corresponding to airport code.
        - `i94port_state_code`: the state the city belong to.
    - `I94MODE_sas_label_validation`
        - `i94mode_valid_code`: Immigration mode code
        - `i94mode_valid_value`: Immigration mode value
    - `I94VISA_sas_label_validation`
        - `i94visa_valid_code`: Code of visa type of travaller
        - `i94visa_valid_value`: Value of visa type of travller
    - `I94ADDR_sas_label_validation`
        - `i94addr_valid_code`: Immigration state code
        - `i94addr_valid_value`: Immigration state value
    - `I94RES_sas_label_validation`
        - `i94res_valid_code`: The country code that traveller come from
        - `i94res_valid_value`: The country value that traveller come from

The perform outputs:
- Amount of records and data size: 
    - I94 Immigration Dataset: `3096313 rows`
    - World Temperature Dataset: `8599212 rows`
    - i94port SAS Labels Dataset: `660 rows`
- Data file extension formats included: 
    - I94 Immigration Dataset is a `.sas7bdat`
    - World Temperature Dataset is a `.csv`
    - SAS Labels Descriptions is a `.SAS`

Our choosen datesets sastify the project rubric and will be using for data modeling.

Our expectations :
- The choosen datasets enough to perform a data modeling of fact and dimention tables to analysis the relationship between amount of travel immigration and weather duration by month of city.
- Visa to city
- Airline traffic
- Immigration age

### Step 2: Explore and Assess the Data

#### Explore the Data

In [1]:
# Do all imports and installs here - Done
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StructType as R, StructField as Fld,\
    DoubleType as Dbl, StringType as Str, IntegerType as Int,\
    TimestampType as Timestamp, DateType as Date, LongType as Long
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
import pandas as pd
import re
import configparser
import os
import shutil
from pathlib import Path
from datetime import datetime

In [2]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
config = configparser.ConfigParser()
config.read('etl.cfg')

os.environ["AWS_ACCESS_KEY_ID"] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"] = config['AWS']['AWS_SECRET_ACCESS_KEY']
AWS_ACCESS_KEY_ID = config['AWS']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['AWS']['AWS_SECRET_ACCESS_KEY']


spark = SparkSession.builder\
        .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0,saurfang:spark-sas7bdat:2.0.0-s_2.11")\
        .config("spark.hadoop.fs.s3a.access.key",AWS_ACCESS_KEY_ID)\
        .config("spark.hadoop.fs.s3a.secret.key",AWS_SECRET_ACCESS_KEY)\
        .enableHiveSupport().getOrCreate()

In [3]:
# Procedure read out validation pair values from SAS Labels Description
def get_validation_code_from_SAS_labels(sas_input_label):
    '''
    This procedure read a input SAS Labels Description and then write out validation code datasets.
    The SAS Labels Description included validation code datasets with labels: I94RES (same to I94CIT), I94PORT, I94ADDR, I94MODE, I94VISA.
    
    Parameters
    ----------
    sas_input_label : string
        The label name of validation code dataset. Its can be one of I94RES (same to I94CIT), I94PORT, I94ADDR, I94MODE, I94VISA.
    
    Returns
    -------
    validation_code_list : validation_value_pairs(tuple(str_valid_code, str_valid_value))
        The return output is a specific SAS label list of validation code value pairs.
    '''

    # Read input SAS Labels Descriptions
    with open('I94_SAS_Labels_Descriptions.SAS') as sas_validation_code:
            labels_from_sas = sas_validation_code.read()

    # Parse labels from SAS Label Description input
    sas_labels = labels_from_sas[labels_from_sas.index(sas_input_label):]
    sas_labels = sas_labels[:sas_labels.index(';')]
    
    # Processing line by line, remove separate charaters and then append value pair
    lines = sas_labels.splitlines()
    validation_code_list = []
    for line in lines:
        try:
            valid_code, valid_value = line.split('=')
            valid_code = valid_code.strip().strip("'").strip('"')
            valid_value = valid_value.strip().strip("'").strip('"').strip()
            validation_code_list.append((valid_code, valid_value))
        except:
            pass
        
    return validation_code_list

In [4]:
# Procedure extract parts from SAS Labels Description
def extract_staging_sas_label(label):
    '''
    This procedure get a specific part of data dictionary from SAS_Labels_Descriptions.SAS to a schema.
    The dictionary part include valid_code and valid_value
    
    Parameters
    ----------
    label: 
        a string input of specific label from "SAS_Label_Descriptions.SAS"
        
    Syntax note: 
        input value in string datatype, need inside a pair of single quotes. Ex: 'I94RES', 'I94PORTS'
    
    Returns
    -------
    Dir of csv files with a specific part as input label.
    '''
    label_name = label
    valid_code = label + "_valid_code"
    valid_value = label + "_valid_value"
    csv_output = label + "_sas_label_validation"
    
    # Create dir for output
    parent_dir = "./"
    path = os.path.join(parent_dir, csv_output)
    try:
        os.makedirs(path, exist_ok = True)
        print("Directory '%s' created successfully" % csv_output)
    except OSError as error:
        print("Directory '%s' can not be created" % csv_output)

    # Define output dataframe structure
    schema = R([
        Fld(valid_code, Str()),
        Fld(valid_value, Str())
    ])

    # Create dataframe from extracted label
    df = spark.createDataFrame(
        data=get_validation_code_from_SAS_labels(label_name),
        schema=schema
    )

    # df.write.mode('overwrite').csv(csv_output)
    shutil.rmtree(csv_output, ignore_errors=False, onerror=None)
    df.write.options(header='True', delimiter=',').csv(csv_output)

    df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv(csv_output)

    print("Top 20 rows of {} ".format(csv_output))
    df.show()

    print("Count rows of {}: {} ".format(csv_output, df.count()))
    
    print("Check unique value of {}: {} ".format(csv_output, df.select(valid_code).distinct().count()))

    print("Staging csv files in: {}".format(csv_output))

    return df

In [5]:
# Procedure convert column name
def convert_column_names(df):
    '''
    This procedure standardizing column names to snake case format. Format ex: customer_name, billing_address, total_price.
    
    Parameters
    ----------
    dataframe : string_of_dataframe
        The input dataframe with column names might have elements of messy columns names, including accents, different delimiters, casing and multiple white spaces.
        Snake case style replaces the white spaces and symbol delimiters with underscore and converts all characters to lower case
    
    Returns
    -------
    Dataframe with column names has been changed to snake_case format.
    '''
    cols = df.columns
    column_name_changed = []

    for col in cols:
        new_column = col.lstrip().rstrip().lower().replace (" ", "_").replace ("-", "_")
        column_name_changed.append(new_column)

    df.columns = column_name_changed

In [6]:
# Procedure remove specific dir (if need)
def rmdir(directory):
    '''
    This procedure perform pure recursive a directory.
    
    Parameters
    ----------
    directory : string_of_path_to_dir
        The input directory is a path to target dir. This dir and all its belong child objects wil be deleted.
        Syntax note: rmdir(Path("target_path_to_dir"))
            with Path("target_path_to_dir") returns path to dir format as 'directory' input
    
    Returns
    -------
    None
    '''
    directory = Path(directory)
    for item in directory.iterdir():
        if item.is_dir():
            rmdir(item)
        else:
            item.unlink()
    directory.rmdir()

##### I94 Immigration dataset

In [7]:
# Read input dataset to Spark dataframe
#i94immi_dataset = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
#i94immi_df = spark.read.format('com.github.saurfang.sas.spark').load(i94immi_dataset)
i94immi_df = spark.read.parquet("sas_data")

In [ ]:
# Data type
i94immi_df.dtypes

In [ ]:
# Attributes columns
i94immi_df.show(5)

In [ ]:
i94immi_df.count()

##### World Temperature dataset

In [8]:
# read input dataset to Pandas dataframe
worldtempe_dataset = '../../data2/GlobalLandTemperaturesByCity.csv'
worldtempe_df = pd.read_csv(worldtempe_dataset,sep=",")

In [ ]:
# Data type
worldtempe_df.info()

In [ ]:
# Attributes columns
worldtempe_df.head(5)

In [ ]:
worldtempe_df.shape

##### I94PORT_sas_label_validation

In [9]:
# Extract `I94PORT` label
# Create dir 'I94PORT_sas_label_validation' to save extracted label result
I94PORT_df = extract_staging_sas_label('I94PORT')
I94PORT_df = I94PORT_df.toPandas()

Directory 'I94PORT_sas_label_validation' created successfully
Top 20 rows of I94PORT_sas_label_validation 
+------------------+--------------------+
|I94PORT_valid_code| I94PORT_valid_value|
+------------------+--------------------+
|               ALC|           ALCAN, AK|
|               ANC|       ANCHORAGE, AK|
|               BAR|BAKER AAF - BAKER...|
|               DAC|   DALTONS CACHE, AK|
|               PIZ|DEW STATION PT LA...|
|               DTH|    DUTCH HARBOR, AK|
|               EGL|           EAGLE, AK|
|               FRB|       FAIRBANKS, AK|
|               HOM|           HOMER, AK|
|               HYD|           HYDER, AK|
|               JUN|          JUNEAU, AK|
|               5KE|       KETCHIKAN, AK|
|               KET|       KETCHIKAN, AK|
|               MOS|MOSES POINT INTER...|
|               NIK|         NIKISKI, AK|
|               NOM|             NOM, AK|
|               PKC|     POKER CREEK, AK|
|               ORI|  PORT LIONS SPB, AK|
|          

In [ ]:
# Data type
I94PORT_df.info()

In [ ]:
# Attributes columns
I94PORT_df.head(5)

In [ ]:
# Count rows
I94PORT_df.count()

##### I94ADDR_sas_label_validation

In [10]:
# Extract `I94ADDR` label
# Create dir 'I94ADDR_sas_label_validation' first to save extracted label result
I94ADDR_df = extract_staging_sas_label('I94ADDR')
I94ADDR_df = I94ADDR_df.toPandas()

Directory 'I94ADDR_sas_label_validation' created successfully
Top 20 rows of I94ADDR_sas_label_validation 
+------------------+-------------------+
|I94ADDR_valid_code|I94ADDR_valid_value|
+------------------+-------------------+
|                AL|            ALABAMA|
|                AK|             ALASKA|
|                AZ|            ARIZONA|
|                AR|           ARKANSAS|
|                CA|         CALIFORNIA|
|                CO|           COLORADO|
|                CT|        CONNECTICUT|
|                DE|           DELAWARE|
|                DC|  DIST. OF COLUMBIA|
|                FL|            FLORIDA|
|                GA|            GEORGIA|
|                GU|               GUAM|
|                HI|             HAWAII|
|                ID|              IDAHO|
|                IL|           ILLINOIS|
|                IN|            INDIANA|
|                IA|               IOWA|
|                KS|             KANSAS|
|                KY|           K

In [ ]:
# Data type
I94ADDR_df.info()

In [ ]:
# Attributes columns
I94ADDR_df.head(5)

In [ ]:
# Count rows
I94ADDR_df.count()

#### Cleaning Steps

##### I94 Immigration dataset

- Cleaning I94 Immigration dataset of a month `../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat`
- The input dataframe from Explorer step `i94immi_df = spark.read.parquet("sas_data")`
- The outputs of this step: `i94immi_df_clean.csv`
- Cleaning task to do:
    - Read dataset to spark dataframe.
    - Create Spark SQL table from dataframe.
    - Choose Primarykey.
    - Verify arrival date and departure date logical conditional.
    - Add column `arival_date`, `departure_date` as `datetime` datatype
    - Verify `arival_date`, `departure_date` wrong value.
    - Filter US airport with immigration allowed.
    - Remove missing value.
    - Create and verify staging table.

In [11]:
i94immi_df.show(3)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

In [11]:
# Create tableview for data manipulation
i94immi_df.createOrReplaceTempView('i94immi_table')

In [12]:
spark.sql("""
    SELECT cicid
    FROM i94immi_table
""").show(3)

+---------+
|    cicid|
+---------+
|5748517.0|
|5748518.0|
|5748519.0|
+---------+
only showing top 3 rows



In [13]:
# Drop amount of un-makesance records cause by `DepartureDate >= ArrivalDate`
spark.sql("""
    SELECT *
    FROM i94immi_table
    WHERE arrdate <= depdate
""").createOrReplaceTempView("i94immi_table")

In [14]:
# Add column `arrival_date = timestone + arrdate_offset_day`, with:
# - timestone = '1960-01-01' (***datetime*** datatype)
# - arrdate_offset_day = 'arrdate' (***integer*** datatype)
# - arrival_date (***datetime*** datatype)
spark.sql("""
    SELECT *, date_add(to_date('1960-01-01'), arrdate) AS arrival_date 
    FROM i94immi_table
""").createOrReplaceTempView("i94immi_table")

In [15]:
# Add column `departure_date = timestone + depdate_offset_day`, with:
# - `timestone` = '1960-01-01' (***datetime*** datatype)
# - `depdate_offset_day` = 'depdate' (***integer*** datatype)
# - `departure_date` (***datetime*** datatype)
spark.sql("""SELECT *, CASE 
                        WHEN depdate >= arrdate THEN date_add(to_date('1960-01-01'), depdate)
                        WHEN depdate IS NULL THEN NULL
                        ELSE 'NaN' END AS departure_date 
                FROM i94immi_table
            """).createOrReplaceTempView("i94immi_table")

In [16]:
# extracted i94mode from `I94_SAS_Labels_Descriptions_SAS`
# i94mode includes:
# {'1': 'Air', '2': 'Sea', '3': 'Land', '9': 'Not reported'}
# Keep air arrival only, mean keep `i94mode=1`
spark.sql("""
    SELECT *
    FROM i94immi_table
    WHERE i94mode == 1.0
""").createOrReplaceTempView("i94immi_table")

In [17]:
# Mapping `i94visa` numbers to `visatype` instead
spark.sql("""
        SELECT *, CASE 
                    WHEN i94visa = 1.0 THEN 'Business' 
                    WHEN i94visa = 2.0 THEN 'Pleasure'
                    WHEN i94visa = 3.0 THEN 'Student'
                    ELSE 'NaN' END AS visa_type
        FROM i94immi_table
    """).createOrReplaceTempView("i94immi_table")

In [18]:
# Keep user records of `male = 'M'` and `female = 'F'` only
spark.sql("""
    SELECT * 
    FROM i94immi_table 
    WHERE gender IN ('F', 'M')
""").createOrReplaceTempView("i94immi_table")

In [19]:
# Drop NULL value on arrival state
spark.sql("""
    SELECT *
    FROM i94immi_table
    WHERE i94addr IS NOT NULL
""").createOrReplaceTempView("i94immi_table")

In [20]:
# Keep necessary columns
# Convert month and year timestamp
spark.sql("""
        SELECT 
            cicid,
            i94cit,
            i94res,
            i94port,
            arrival_date,
            YEAR(arrival_date) as i94yr,
            MONTH(arrival_date) as i94mon,
            i94mode,
            i94addr,
            departure_date,
            i94bir,
            i94visa,
            count,
            dtadfile,
            biryear,
            dtaddto,
            gender,
            insnum,
            airline,
            admnum,
            fltno,
            visatype,
            visa_type
        FROM i94immi_table
            """).createOrReplaceTempView('i94immi_table')

In [21]:
# Verify the cleaned
i94immi_df = spark.sql("""
    SELECT *
    FROM i94immi_table
""")

In [22]:
i94immi_df.show(3)

+---------+------+------+-------+------------+-----+------+-------+-------+--------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|    cicid|i94cit|i94res|i94port|arrival_date|i94yr|i94mon|i94mode|i94addr|departure_date|i94bir|i94visa|count|dtadfile|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|visa_type|
+---------+------+------+-------+------------+-----+------+-------+-------+--------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|5748517.0| 245.0| 438.0|    LOS|  2016-04-30| 2016|     4|    1.0|     CA|    2016-05-08|  40.0|    1.0|  1.0|20160430| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1| Business|
|5748518.0| 245.0| 438.0|    LOS|  2016-04-30| 2016|     4|    1.0|     NV|    2016-05-17|  32.0|    1.0|  1.0|20160430| 1984.0|10292016|     F|  null|     VA|9.495562283E10|00007|      B1| Business|


In [23]:
# Verify the cleaned
spark.sql("""
    SELECT COUNT(*) as amount_i94immi_rows
    FROM i94immi_table
""").show()

+-------------------+
|amount_i94immi_rows|
+-------------------+
|            2377896|
+-------------------+



In [24]:
# Staging to csv file
path = './i94immi_df_clean'
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % path)
except OSError as error:
    print("Directory '%s' can not be created" % path)

rmdir(Path("i94immi_df_clean")) # use this line from 2nd running
i94immi_df.write.options(header='True', delimiter=',').csv("i94immi_df_clean")

Directory './i94immi_df_clean' created successfully


In [25]:
# Verify out from staging csv partitions
i94immi_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94immi_df_clean")

In [26]:
# Prepare to save out s3 storage
i94immi_df.show(3)

+---------+------+------+-------+-------------------+-----+------+-------+-------+-------------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|    cicid|i94cit|i94res|i94port|       arrival_date|i94yr|i94mon|i94mode|i94addr|     departure_date|i94bir|i94visa|count|dtadfile|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|visa_type|
+---------+------+------+-------+-------------------+-----+------+-------+-------+-------------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|5748517.0| 245.0| 438.0|    LOS|2016-04-30 00:00:00| 2016|     4|    1.0|     CA|2016-05-08 00:00:00|  40.0|    1.0|  1.0|20160430| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1| Business|
|5748518.0| 245.0| 438.0|    LOS|2016-04-30 00:00:00| 2016|     4|    1.0|     NV|2016-05-17 00:00:00|  32.0|    1.0|  1.0|20160430| 1984.0|10292016|   

In [40]:
i94immi_df.dtypes

[('cicid', 'double'),
 ('i94cit', 'double'),
 ('i94res', 'double'),
 ('i94port', 'string'),
 ('arrival_date', 'timestamp'),
 ('i94yr', 'int'),
 ('i94mon', 'int'),
 ('i94mode', 'double'),
 ('i94addr', 'string'),
 ('departure_date', 'timestamp'),
 ('i94bir', 'double'),
 ('i94visa', 'double'),
 ('count', 'double'),
 ('dtadfile', 'int'),
 ('biryear', 'double'),
 ('dtaddto', 'string'),
 ('gender', 'string'),
 ('insnum', 'string'),
 ('airline', 'string'),
 ('admnum', 'double'),
 ('fltno', 'string'),
 ('visatype', 'string'),
 ('visa_type', 'string')]

##### World Temperature dataset

- Cleaning World Temperature dataset `../../data2/GlobalLandTemperaturesByCity.csv`
- The input dataframe from Explorer step: `worldtempe_df = pd.read_csv(worldtempe_dataset,sep=",")`
- The outputs of this step: `worldtempe_df_clean.csv`.
- Cleaning task to do:
    - Read dataset to pandas dataframe.
    - Filter value of `United States` only.
    - Limit dataset duration by immigration time duration
    - Clean column with datetime datatype.
    - Standalizing column names format.
    - Create and verify staging table.

In [27]:
# Filter out `Country` for single value `United States` and check dataframe size
worldtempe_df = worldtempe_df[worldtempe_df['Country']=='United States']

In [28]:
# Add a column 'dt_converted' as datetime datatype
worldtempe_df['dt_converted'] = pd.to_datetime(worldtempe_df.dt)

In [29]:
# Cut off the sub-dataset before "1960-01-01"
worldtempe_df=worldtempe_df[worldtempe_df['dt_converted']>"1960-01-01"]

In [30]:
# Upper case 'City' column values
worldtempe_df["City"] = worldtempe_df["City"].str.upper()

In [31]:
# Convert column names to ***snake_case*** format. Format ex: *customer_name, billing_address, ...*
convert_column_names(worldtempe_df)
worldtempe_df.columns

Index(['dt', 'averagetemperature', 'averagetemperatureuncertainty', 'city',
       'country', 'latitude', 'longitude', 'dt_converted'],
      dtype='object')

In [32]:
# Staging to csv file (optional)
worldtempe_df.to_csv('worldtempe_df_clean.csv', index=False, header=True)

In [33]:
# Verify out from staging csv partitions (optional)
# worldtempe_df = pd.read_csv('worldtempe_df_clean.csv',sep=",")
worldtempe_df = spark.read.csv("worldtempe_df_clean.csv", header=True)

In [48]:
worldtempe_df.dtypes

[('dt', 'string'),
 ('averagetemperature', 'string'),
 ('averagetemperatureuncertainty', 'string'),
 ('city', 'string'),
 ('country', 'string'),
 ('latitude', 'string'),
 ('longitude', 'string'),
 ('dt_converted', 'string')]

In [34]:
worldtempe_df.show(5)

+----------+------------------+-----------------------------+-------+-------------+--------+---------+------------+
|        dt|averagetemperature|averagetemperatureuncertainty|   city|      country|latitude|longitude|dt_converted|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+------------+
|1960-02-01|             4.995|                        0.325|ABILENE|United States|  32.95N|  100.53W|  1960-02-01|
|1960-03-01| 8.575000000000001|                        0.303|ABILENE|United States|  32.95N|  100.53W|  1960-03-01|
|1960-04-01|            18.452|                        0.282|ABILENE|United States|  32.95N|  100.53W|  1960-04-01|
|1960-05-01|            21.709|          0.28600000000000003|ABILENE|United States|  32.95N|  100.53W|  1960-05-01|
|1960-06-01|            27.714|                        0.387|ABILENE|United States|  32.95N|  100.53W|  1960-06-01|
+----------+------------------+-----------------------------+-------+---

In [36]:
# Change datatype of column '' as DoubleType datatype for average temperature of eache city
worldtempe_df = worldtempe_df.withColumn("averagetemperature", worldtempe_df["averagetemperature"].cast(DoubleType()).alias("averagetemperature"))

In [37]:
# Lamda function to convert string of date format to date datatype
func =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d'), DateType())

In [38]:
# Convert 'dt_converted' to date datatype
worldtempe_df = worldtempe_df.withColumn('dt_converted', func(col('dt_converted')))

In [39]:
# Create a tableview for Spark SQL dataframe manipulation
worldtempe_df.createOrReplaceTempView('worldtempe_table')

In [40]:
# Add columns 'tempe_month' as date and 'tempe_year' as date
spark.sql("""
        SELECT 
            dt_converted,
            MONTH(worldtempe_table.dt_converted) as tempe_month,
            YEAR(worldtempe_table.dt_converted) as tempe_year,
            dt,
            city,
            averagetemperature,
            averagetemperatureuncertainty
        FROM worldtempe_table
            """).createOrReplaceTempView('worldtempe_table')

In [41]:
# Grouping average temperature by city for city temperature consistent
spark.sql("""
    SELECT 
        city,
        tempe_month,
        BROUND(AVG(averagetemperature),2) as averagetemperature,
        BROUND(AVG(averagetemperatureuncertainty),2) as averagetemperatureuncertainty,
        tempe_year,
        dt_converted
    FROM worldtempe_table
    GROUP BY city, tempe_month, tempe_year, dt_converted
""").createOrReplaceTempView('worldtempe_table')

In [42]:
worldtempe_df = spark.sql("""
    SELECT *
    FROM worldtempe_table
""")

In [43]:
# Staging to csv file
path = './worldtempe_df_clean'
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % path)
except OSError as error:
    print("Directory '%s' can not be created" % path)
    
rmdir(Path("worldtempe_df_clean")) # use this line from 2nd running
worldtempe_df.write.options(header='True', delimiter=',').csv("worldtempe_df_clean")

Directory './worldtempe_df_clean' created successfully


In [44]:
# Verify out from staging csv partitions
worldtempe_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("worldtempe_df_clean")

In [45]:
# Verify csv staging
worldtempe_df.show(5)

+-------+-----------+------------------+-----------------------------+----------+-------------------+
|   city|tempe_month|averagetemperature|averagetemperatureuncertainty|tempe_year|       dt_converted|
+-------+-----------+------------------+-----------------------------+----------+-------------------+
|ABILENE|         11|              9.77|                         0.12|      1997|1997-11-01 00:00:00|
|ABILENE|         10|             18.48|                         0.26|      2000|2000-10-01 00:00:00|
|  AKRON|          2|              2.17|                         0.18|      1976|1976-02-01 00:00:00|
|  AKRON|          9|              17.5|                         0.22|      1990|1990-09-01 00:00:00|
|  AKRON|          7|             24.95|                         0.21|      2011|2011-07-01 00:00:00|
+-------+-----------+------------------+-----------------------------+----------+-------------------+
only showing top 5 rows



In [66]:
# Verify csv staging
worldtempe_df.dtypes

[('city', 'string'),
 ('tempe_month', 'int'),
 ('averagetemperature', 'double'),
 ('averagetemperatureuncertainty', 'double'),
 ('tempe_year', 'int'),
 ('dt_converted', 'timestamp')]

##### I94PORT_sas_label_validation

- Cleaning `i94port` from `I94_SAS_Labels_Descriptions.SAS`
- The input dataframe from Explorer step: `I94PORT_df = I94PORT_df.toPandas()`
- The outputs of this step: `i94_port.csv`
- Cleaning task to do: 
    - Extract `I94PORT` from `I94_SAS_Labels_Descriptions.SAS`
    - Clean leading and trailing white space.
    - Split to port_code, city, state.
    - Clean others columns to limit dataset.
    - Create and verify staging table.

In [46]:
# clean leading and trailing white space before split column
I94PORT_df["I94PORT_valid_code"] = I94PORT_df["I94PORT_valid_code"].str.lstrip().str.rstrip()
I94PORT_df["I94PORT_valid_value"] = I94PORT_df["I94PORT_valid_value"].str.lstrip().str.rstrip()

In [47]:
# split to port, city, state
I94PORT_df["I94PORT_city_name"] = I94PORT_df["I94PORT_valid_value"].str.split(",").str.get(0)
I94PORT_df["I94PORT_state_code"] = I94PORT_df["I94PORT_valid_value"].str.split(",").str.get(1)

In [48]:
# clean leading and trailing white space after split column
I94PORT_df["I94PORT_city_name"] = I94PORT_df["I94PORT_city_name"].str.lstrip().str.rstrip()
I94PORT_df["I94PORT_state_code"] = I94PORT_df["I94PORT_state_code"].str.lstrip().str.rstrip()

In [49]:
# drop missing value on I94PORT_state_code
I94PORT_df = I94PORT_df.dropna(subset = ["I94PORT_state_code"])

In [50]:
# Convert column name to snake case format
convert_column_names(I94PORT_df)

In [51]:
I94PORT_df.columns

Index(['i94port_valid_code', 'i94port_valid_value', 'i94port_city_name',
       'i94port_state_code'],
      dtype='object')

In [52]:
# Remove 'i94port_valid_value' column
select_cols = ['i94port_valid_code', 'i94port_city_name', 'i94port_state_code']
I94PORT_df = I94PORT_df[select_cols]

In [53]:
I94PORT_df.head(3)

,i94port_valid_code,i94port_city_name,i94port_state_code
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK


In [54]:
# Changing to SQL Spark dataframe for staging output saving
I94PORT_df = spark.createDataFrame(I94PORT_df)

In [55]:
# Save to csv staging
path = './i94port_staging'
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % path)
except OSError as error:
    print("Directory '%s' can not be created" % path)
    
rmdir(Path("i94port_staging")) # use from 2nd running
I94PORT_df.write.options(header='True', delimiter=',').csv("i94port_staging")

Directory './i94port_staging' created successfully


In [56]:
# Read csv staging to create dim table
I94PORT_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94port_staging")

In [57]:
I94PORT_df.show(3)

+------------------+--------------------+------------------+
|i94port_valid_code|   i94port_city_name|i94port_state_code|
+------------------+--------------------+------------------+
|               ALC|               ALCAN|                AK|
|               ANC|           ANCHORAGE|                AK|
|               BAR|BAKER AAF - BAKER...|                AK|
+------------------+--------------------+------------------+
only showing top 3 rows



##### I94ADDR_sas_label_validation

- Cleaning `i94addr` from `I94_SAS_Labels_Descriptions.SAS`
- The input dataframe from Explorer step: `I94ADDR_df = I94ADDR_df.toPandas()`
- The outputs of this step: `i94_addr.csv`
- Cleaning task to do: 
    - Extract `I94PORT` from `I94_SAS_Labels_Descriptions.SAS`
    - Clean leading and trailing white space.
    - Distinct primarykey columns.
    - Create and verify staging table.

In [58]:
# clean leading and trailing white space before split column
I94ADDR_df["I94ADDR_valid_code"] = I94ADDR_df["I94ADDR_valid_code"].str.lstrip().str.rstrip()
I94ADDR_df["I94ADDR_valid_value"] = I94ADDR_df["I94ADDR_valid_value"].str.lstrip().str.rstrip()

In [59]:
# Convert column name to snake case format
convert_column_names(I94ADDR_df)

In [60]:
# Changing to SQL Spark dataframe for staging output saving
I94ADDR_df = spark.createDataFrame(I94ADDR_df)

In [61]:
# Save to csv staging
path = './i94addr_staging'
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % path)
except OSError as error:
    print("Directory '%s' can not be created" % path)
    
rmdir(Path("i94addr_staging")) # use from 2nd running
I94ADDR_df.write.options(header='True', delimiter=',').csv('i94addr_staging')

Directory './i94addr_staging' created successfully


In [62]:
# Read csv staging to create dim table
I94ADDR_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94addr_staging")

In [63]:
I94ADDR_df.show(3)

+------------------+-------------------+
|i94addr_valid_code|i94addr_valid_value|
+------------------+-------------------+
|                AL|            ALABAMA|
|                AK|             ALASKA|
|                AZ|            ARIZONA|
+------------------+-------------------+
only showing top 3 rows



### Step 3: Define the Data Model

#### 3.1 Conceptual Data Model

As expectation mention, we want to find out the relations between US immigration with either weather, immigration traffic and the arrival place (city). To archive the expectation, we create star data modeling with fact and dim tables detail bellow:
Airport
Traffic <note_here>

Start schema diagram transformed
- Start_schema_diagram here

##### I94 Immigration fact table:

Input dataframe come from 
`i94immi_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94immi_df_clean")`

- `fact_i94immi` should includes columns:
    - `travel_cicid`
    - `from_country_code`
    - `immi_country_code`
    - `arrival_port_code`
    - `immi_arrival_date`
    - `arrival_year`
    - `arrival_month`
    - `airline_mode_code`
    - `immi_state_code`
    - `departure_date`
    - `traveller_age`
    - `visatype_by_number`
    - `traveller_birth_year`
    - `traveller_sex`
    - `immi_flight_code`
    - `visatype_by_code`
    - `visa_type`


In [83]:
i94immi_df.show(3)

+---------+------+------+-------+-------------------+-----+------+-------+-------+-------------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|    cicid|i94cit|i94res|i94port|       arrival_date|i94yr|i94mon|i94mode|i94addr|     departure_date|i94bir|i94visa|count|dtadfile|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|visa_type|
+---------+------+------+-------+-------------------+-----+------+-------+-------+-------------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|5748517.0| 245.0| 438.0|    LOS|2016-04-30 00:00:00| 2016|     4|    1.0|     CA|2016-05-08 00:00:00|  40.0|    1.0|  1.0|20160430| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1| Business|
|5748518.0| 245.0| 438.0|    LOS|2016-04-30 00:00:00| 2016|     4|    1.0|     NV|2016-05-17 00:00:00|  32.0|    1.0|  1.0|20160430| 1984.0|10292016|   

In [64]:
# Change to Spark SQl
i94immi_df.createOrReplaceTempView('fact_i94immi')

In [65]:
# Create fact table
fact_i94immi = spark.sql("""
        SELECT
            cicid as travel_cicid,
            i94cit as from_country_code,
            i94res as immi_country_code,
            i94port as arrival_port_code,
            arrival_date as immi_arrival_date,
            i94yr as arrival_year,
            i94mon as arrival_month,
            i94mode as airline_mode_code,
            i94addr as immi_state_code,
            departure_date,
            i94bir as traveller_age,
            i94visa as visatype_by_number,
            biryear as traveller_birth_year,
            gender as traveller_sex,
            fltno as immi_flight_code,
            visatype as visatype_by_code,
            visa_type
        FROM fact_i94immi
    """)

In [66]:
# Copy S3 URI = 's3a://dendcapstonetest/s3_outputs/'
# update this to your bucket name
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'fact_i94immi/fact_i94immi.parquet'
# s3_parquet_output = './ws_outputs' + 'fact_i94immi/fact_i94immi.parquet'
# s3_parquet_output

parquet_outputs = './ws_parquet_outputs/' + 'fact_i94immi.parquet'
parquet_outputs

'./ws_parquet_outputs/fact_i94immi.parquet'

In [67]:
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'fact_i94immi/fact_i94immi.parquet'
# parquet_outputs = ./ws_parquet_outputs/' + 'fact_i94immi.parquet'
fact_i94immi.write.mode("overwrite").parquet(parquet_outputs)

##### Visa dim table

Input dataframe come from 
`i94immi_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94immi_df_clean")`

- `dim_visa` should includes columns:
    - `visatype_by_number`
    - `visatype_by_code`
    - `visa_category`

In [68]:
# Create Spark SQL for dataframe manipulation
i94immi_df.createOrReplaceTempView('dim_visa')

In [69]:
# Create dim table
dim_visa = spark.sql("""
        SELECT
            i94visa as visatype_by_number,
            visatype as visatype_by_code,
            visa_type as visa_category
        FROM fact_i94immi
    """)

In [71]:
# Copy S3 URI = 's3a://dendcapstonetest/s3_outputs/'
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'dim_visa/dim_visa.parquet'
parquet_outputs = './ws_parquet_outputs/' + 'dim_visa.parquet'
parquet_outputs

'./ws_parquet_outputs/dim_visa.parquet'

In [72]:
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'fact_i94immi/fact_i94immi.parquet'
dim_visa.write.mode("overwrite").parquet(parquet_outputs)

##### Flight dim table:

Input dataframe come from 
`i94immi_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94immi_df_clean")`

- `dim_immi_flight` should includes columns:
    - `flight_brand`
    - `flight_number`

In [73]:
# Create Spark SQL for dataframe manipulation
i94immi_df.createOrReplaceTempView('dim_immi_flight')

In [74]:
# Create dim table
dim_immi_flight = spark.sql("""
        SELECT
            airline as flight_brand,
            fltno as flight_number
        FROM dim_immi_flight
    """)

In [75]:
# Copy S3 URI = 's3a://dendcapstonetest/s3_outputs/'
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'dim_immi_flight/dim_immi_flight.parquet'
parquet_outputs = './ws_parquet_outputs/' + 'dim_immi_flight.parquet'
parquet_outputs

'./ws_parquet_outputs/dim_immi_flight.parquet'

In [76]:
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'fact_i94immi/fact_i94immi.parquet'
dim_immi_flight.write.mode("overwrite").parquet(parquet_outputs)

##### Immigration travller dim table

Input dataframe come from 
`i94immi_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94immi_df_clean")`

- `dim_immi_travaller` should includes columns:
    - `traveller_cicid`
    - `from_country_code`
    - `immi_country_code`
    - `arrival_port_code`
    - `arrival_date`
    - `airline_immi_code`
    - `desination_state_code`
    - `traveller_age`
    - `traveller_birth_year`
    - `traveller_sex`
    - `visatye_by_code`

In [77]:
i94immi_df.createOrReplaceTempView('dim_immi_travaller')

In [80]:
dim_immi_travaller = spark.sql("""
        SELECT
            cicid as traveller_cicid,
            i94cit as from_country_code,
            i94res as immi_country_code,
            i94port as arrival_port_code,
            arrival_date as arrival_date,
            i94mode as airline_immi_code,
            i94addr as desination_state_code,
            i94bir as traveller_age,
            biryear as traveller_birth_year,
            gender as traveller_sex,
            visatype as visatye_by_code
        FROM dim_immi_travaller
    """)

In [81]:
# Copy S3 URI = 's3a://dendcapstonetest/s3_outputs/'
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'dim_immi_travaller/dim_immi_travaller.parquet'
parquet_outputs = './ws_parquet_outputs/' + 'dim_immi_travaller.parquet'
parquet_outputs

'./ws_parquet_outputs/dim_immi_travaller.parquet'

In [82]:
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'fact_i94immi/fact_i94immi.parquet'
dim_immi_travaller.write.mode("overwrite").parquet(parquet_outputs)

##### World Temperature fact table

Input dataframe `worldtempe_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("worldtempe_df_clean")`

- `fact_us_temperature` contains temperature records of US cities has been collect corresponse immigration data scope.
    - `dt`
    - `averagetemperature as avg_tempe`
    - `averagetemperatureuncertainty as avg_tempe_uncertain`
    - `city as measure_city`
    - `country as measure_country`
    - `latitude`
    - `longitude`
    - `dt_converted as measure_date`

In [83]:
# Create tableview for data manipulation
worldtempe_df.createOrReplaceTempView('fact_worldtempe')

In [84]:
spark.sql("""
        SELECT *
        FROM fact_worldtempe
    """).show(3)

+-------+-----------+------------------+-----------------------------+----------+-------------------+
|   city|tempe_month|averagetemperature|averagetemperatureuncertainty|tempe_year|       dt_converted|
+-------+-----------+------------------+-----------------------------+----------+-------------------+
|ABILENE|         11|              9.77|                         0.12|      1997|1997-11-01 00:00:00|
|ABILENE|         10|             18.48|                         0.26|      2000|2000-10-01 00:00:00|
|  AKRON|          2|              2.17|                         0.18|      1976|1976-02-01 00:00:00|
+-------+-----------+------------------+-----------------------------+----------+-------------------+
only showing top 3 rows



In [85]:
# Create fact table
fact_worldtempe = spark.sql("""
        SELECT
            averagetemperature as avg_tempe,
            averagetemperatureuncertainty as avg_tempe_uncertain,
            city as measure_city,
            tempe_month as tempe_month,
            dt_converted as dt_converted,
            dt_converted as measure_date
        FROM fact_worldtempe
    """)

In [86]:
spark.sql("""
    SELECT *
    FROM fact_worldtempe
""").show(3)

+-------+-----------+------------------+-----------------------------+----------+-------------------+
|   city|tempe_month|averagetemperature|averagetemperatureuncertainty|tempe_year|       dt_converted|
+-------+-----------+------------------+-----------------------------+----------+-------------------+
|ABILENE|         11|              9.77|                         0.12|      1997|1997-11-01 00:00:00|
|ABILENE|         10|             18.48|                         0.26|      2000|2000-10-01 00:00:00|
|  AKRON|          2|              2.17|                         0.18|      1976|1976-02-01 00:00:00|
+-------+-----------+------------------+-----------------------------+----------+-------------------+
only showing top 3 rows



In [87]:
# Copy S3 URI = 's3a://dendcapstonetest/s3_outputs/'
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'fact_worldtempe/fact_worldtempe.parquet'
# s3_parquet_output

parquet_outputs = './ws_parquet_outputs/' + 'fact_worldtempe.parquet'
parquet_outputs

'./ws_parquet_outputs/fact_worldtempe.parquet'

In [88]:
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'fact_i94immi/fact_i94immi.parquet'
# fact_worldtempe.write.mode("overwrite").parquet(s3_parquet_output)
fact_worldtempe.write.mode("overwrite").parquet(parquet_outputs)

##### I94PORT dim table

Input dataframe `I94PORT_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94port_staging")`

- `dim_port` contains list of airport allow immigration.
    - `port_code`
    - `city_name`
    - `state`

In [89]:
# Changing to SQL Spark tempview to create dim table
I94PORT_df.createOrReplaceTempView('dim_i94port')

In [90]:
dim_i94port = spark.sql("""
        SELECT
            i94port_valid_code as immi_port_code,
            i94port_city_name as immi_city_name,
            i94port_state_code as immi_state_code
        FROM dim_i94port
    """)

In [91]:
# Copy S3 URI = 's3a://dendcapstonetest/s3_outputs/'
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'dim_i94port/dim_i94port.parquet'
# s3_parquet_output

parquet_outputs = './ws_parquet_outputs/' + 'dim_i94port.parquet'
parquet_outputs

'./ws_parquet_outputs/dim_i94port.parquet'

In [92]:
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'fact_i94immi/fact_i94immi.parquet'
# dim_i94port.write.mode("overwrite").parquet(s3_parquet_output)
dim_i94port.write.mode("overwrite").parquet(parquet_outputs)

##### I94ADDR dim table

Input dataframe `I94ADDR_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94addr_staging")`

- `dim_i94addr` airline, flight number, flight time. (not yet)
    - `immi_state_code` 
    - `immi_state_name`

In [93]:
# Changing to SQL Spark tempview to create dim table
I94ADDR_df.createOrReplaceTempView('dim_i94addr')

In [94]:
dim_i94addr = spark.sql("""
        SELECT
            i94addr_valid_code as immi_state_code,
            i94addr_valid_code as immi_state_name
        FROM dim_i94addr
    """)

In [95]:
# Copy S3 URI = 's3a://dendcapstonetest/s3_outputs/'
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'dim_i94addr/dim_i94addr.parquet'
# s3_parquet_output

parquet_outputs = './ws_parquet_outputs/' + 'dim_i94addr.parquet'
parquet_outputs

'./ws_parquet_outputs/dim_i94addr.parquet'

In [96]:
# s3_parquet_output = 's3a://dendcapstonetest/s3_outputs/' + 'fact_i94immi/fact_i94immi.parquet'
# dim_i94addr.write.mode("overwrite").parquet(s3_parquet_output)
dim_i94addr.write.mode("overwrite").parquet(parquet_outputs)

#### 3.2 Mapping Out Data Pipelines

The pipeline steps are described below:
- Load raw datasources:
    - `./sas_data` contains I94 Immigration datasource.
    - `../../data2/GlobalLandTemperaturesByCity.csv` contains World Temperature datasource.
    - `I94_SAS_Labels_Descriptions.SAS` contains dictionary of `I94PORT` `I94ADDR` `I94VISA` `I94MODE` `I94CIT&RES` datasource.
- Describe and Gather Data on:
    - `i94immi_df` as Spark dataframe.
    - `worldtempe_df` as Pandas dataframe.
    - `I94PORT_df` `I94ADDR_df` `I94VISA_df` `I94MODE_df` as a Spark dataframes.
- Clean and then staging dataframe:
    - `i94immi_df` cleaned and staging as a csv format in dir `i94immi_df_clean`.
    - `worldtempe_df` cleaned and staging as a csv format in dir `worldtempe_df_clean`.
    - `I94PORT` cleaned and staging as a csv format in dir `I94PORT_sas_label_validation`.
    - `I94ADDR` cleaned and staging as a csv format in dir `I94ADDR_sas_label_validation`.
- Transform staging tables to fact and dim tables.
    - `fact_i94immi` out to parquet files saved at `./ws_parquet_outputs/fact_i94immi.parquet`
    - `fact_worldtempe` out to parquet files saved at `./ws_parquet_outputs/fact_worldtempe.parquet`
    - `dim_visa` out to parquet files saved at `./ws_parquet_outputs/dim_visa.parquet`
    - `dim_immi_flight` out to parquet files saved at `./ws_parquet_outputs/dim_immi_flight.parquet`
    - `dim_immi_travaller` out to parquet files saved at `./ws_parquet_outputs/dim_immi_travaller.parquet`
    - `dim_i94port` out to parquet files saved at `./ws_parquet_outputs/dim_i94port.parquet`
    - `dim_i94addr` out to parquet files saved at `./ws_parquet_outputs/dim_i94addr.parquet`
- Create data quality check for fact and dim tables.
    - For dim tables `quality_check_dims.ipynb` (not yet)
    - For fact tables `quality_check_fact.ipynb` (not yet)

### Step 4: Run Pipelines to Model the Data

#### 4.1 Create the data model
Run steps of pipeline with a python

#### 4.2 Data Quality Checks

Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:

* Integrity constraints on the relational database (e.g., unique key, data type, etc.)
* Unit tests for the scripts to ensure they are doing the right thing
* Source/Count checks to ensure completeness

Run Quality Checks

#### 4.3 Data dictionary

- The fact table `fact_immi_weather` should includes columns:
    - `traveller_cicid` as `varchar` datatype
    - `arr_airport_code` as `varchar` datatype
    - `arr_city` as `varchar` datatype
    - `avg_tempe` as `doudbletype` datatype
    - `avg_uncertain_tempe` as `doudbletype` datatype
    - `arr_datetime_iso` as `datetime` datatype
    - `arr_year` as `datetime` datatype
    - `arr_month` as `datetime` datatype
    - `arr_state_code` as `varchar` datatype

Dim tables

- `dim_immi_traveller` contains travller informations like cicid, date, airport, city.
    - `immi_cicid` as `varchar` datatype
    - `immi_datetime_iso` as `datetime` datatype
    - `arr_port_code` as `varchar` datatype
    - `travel_city` as `varchar` datatype
    - `travel_month` as `datetime` datatype
    - `travel_year` as `datetime` datatype

- `dim_us_temperature` contains temperature records of US cities has been collect corresponse immigration data scope.
    - `city_tempe_collect` as `datetime` datatype
    - `avg_tempe` as `doudbletype` datatype
    - `avg_uncertain_tempe` as `doudbletype` datatype
    - `tempe_month` as `datetime` datatype
    - `tempe_year` as `datetime` datatype

- `dim_port` contains list of airport allow immigration.
    - `port_code` as `varchar` datatype
    - `city_name` as `varchar` datatype
    - `state` as `varchar` datatype

- `dim_datetime` contains date information like year, month, day, week of year and weekday.
    - `arrival_year` as `datetime` datatype
    - `arrival_month` as `datetime` datatype
    - `arrival_date` as `datetime` datatype
    * dim_datetime created by append datetime from staging data `i94immi_table`. In this project we use **2016 April** only.

### Step 5: Complete Project Write Up

- Clearly state the rationale for the choice of tools and technologies for the project.
    - Pandas was chosen since it can easily handle dataframe either input or output csv. Easy to install and config Pandas on local computer to push up progress with SDK as VScode or Atom.
    - Spark were chosen since capable of handling support file formats (ex. sas7bdat, SAS) with large volume of data.
    - Dataframes of Spark or Pandas can be converted to each other.
    - Spark SQL was chosen since capable of processing the large input files into dataframes and manipulated via commom SQL JOIN, modify table structure, aggregate data values.

- Propose how often the data should be updated and why.
    - Depending on the purpose of use to make period data analysis suggestions.
        * Yearly: To have data for data analysis education
        * Monthly: To have a basis for predicting seasonal travller traffic.
        * Weekly: To have a basis for serving medical purposes (tracing infectious diseases like COVID, flu, ...).

- Write a description of how you would approach the problem differently under the following scenarios the data was increased by 100x:
    - For the case that need to be met in terms of periodic timing, 100x is a really big volume to meet target be on time. For this scenarios, we should make design a region distributed big data for collecting, processing, modeling. This distributed big data should be many cluster on cloud-based system. Data will be split to chunks, every chunks being processed by a batch-job.
    - For the case no worry about time, we can process the data as a single batch job. We could use existing cloud-based bigdata processing solution as datalake, datawarehouse...

- The data populates a dashboard that must be updated on a daily basis by 7am every day.
    - For specific time update dashboard visualization, we can apply pipeline automation. Airflow is a good candidate cause of capacities schedule, automation, processing batch-job.
    - The configurations of cloud-based processing system is a key factor to speed up the analysis progress.

- The database needed to be accessed by 100+ people.
    - We could consider publishing the parquet files to read-ony HDFS. 
    - In scenarios need  to meet high frequency SQL queries we consider place a midleware (ex. redis, memcahe, memsql) layer in the front of database.